In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout

In [ ]:
from google.colab import files
dataset = files.upload()

In [ ]:
df=pd.read_csv('NKE (2).csv')

In [ ]:
df.shape

In [ ]:
df=df['Close'].values

In [ ]:
df.shape

In [ ]:
df=df.reshape(-1,1) # This makes an array of multiple 1 element arrays for each of the prices

In [ ]:
train=np.array(df[:int(df.shape[0]*0.8)])
test=np.array(df[int(df.shape[0]*0.8):df.shape[0]]) #The test data is the opening prices for 
print(train.shape)
print(test.shape)

In [ ]:
scale=MinMaxScaler(feature_range=(0,1))
train=scale.fit_transform(train)
test=scale.fit_transform(test)

In [ ]:
past=65
def create_dataset(df): #This function creates numpy array of multiple arrays of length 'past' as in our network we will use data from 'past' many days
    x = []
    y = []
    for i in range(past, df.shape[0]):
        x.append(df[i-past:i, 0])
        y.append(df[i, 0])
    x = np.array(x)
    y = np.array(y)
    return x,y

In [ ]:
x_train, y_train = create_dataset(train)
x_test, y_test = create_dataset(test)


In [ ]:
print(x_train)
print(x_train.shape[1])

In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
model = Sequential()
model.add(LSTM(units=120, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=120,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=120,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=120))
model.add(Dropout(0.2)) #dropout is used to prevent overfitting
model.add(Dense(units=1))

In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
model.fit(x_train, y_train, epochs=60, batch_size=50)
model.save('stock_prediction.h5')

In [ ]:
model = load_model('stock_prediction.h5')

In [ ]:


predictions = model.predict(x_test)
predictions = scale.inverse_transform(predictions)
y_test_scaled = scale.inverse_transform(y_test.reshape(-1, 1))

fig, ax = plt.subplots(figsize=(16,8))
ax.set_facecolor('#000030')
ax.plot(y_test_scaled, color='red', label='Original price')
plt.plot(predictions, color='blue', label='Predicted price')
plt.legend()

#print(predictions)
#print('')
#print(y_test_scaled)

In [ ]:
df=pd.read_csv('CEG.L.csv')

In [ ]:
new_df=df['Close'].values

In [ ]:
#Get the last 'past' days to predict something in the future
last_days=new_df[-past:]
last_days=np.reshape(last_days, (-1,1))

In [ ]:
scale_last_days=scale.transform(last_days)

In [ ]:
#Projecting days in future using data that the network generates itself.
#In this case I have projected 20 days into the future
days=20
results=[]
Days=[]
for i in range(days):
  Days.append(i+1)
  Test=[]
  Test.append(scale_last_days[-past:,0])
  Test=np.array(Test)
  Test=np.reshape(Test, (Test.shape[0], Test.shape[1], 1))
  pred=model.predict(Test)
  scale_last_days=np.append(scale_last_days, pred)
  scale_last_days=np.reshape(scale_last_days, (-1,1))
  pred=scale.inverse_transform(pred)
  print(pred)
  
  pred=pred[0][0]
  print(pred)
  results.append(pred)






In [ ]:
fig, ax = plt.subplots(figsize=(16,8))
ax.set_facecolor('#000041')
ax.plot(Days, results, color='cyan', label='predicted price')

In [ ]:
#This code outputs the first predicted day in the future.
results[0]